# 2
# PPFQ

## Construccion de indicadores

-57-

Existen categorias de indicadores:

- Indicadores sobre series de precio
    - indicadores tardíos (trend-following)
    - indicadores de saturacion (contrarians)
    - indicadores de referencia: Osciladores acotados (sobre precio o sobre indicadores)

<br />

- indicadores de flujo (volumen o derivados)

- indicadores combinados

- conteos discretos

- estacionalidad

- indicadores estáticos

- Referenciales (Benchmarks)

- Ratios y series de Análisis fundamental

- Sentiment

- Exógenos al mercado

In [1]:
from dotenv import load_dotenv
import os
from apps.forex_api import  alpha_vantage_fx_api
import pandas as pd
import numpy as np
import seaborn as sns


load_dotenv()
TOKEN = os.environ["TOKEN_AV"]
# data = alpha_vantage_fx_api("FX_DAILY", "EUR", "USD", TOKEN)

In [2]:
# data.to_csv('data-daily-raw.csv')

data = pd.read_csv("data-daily-raw.csv", index_col=0)

In [3]:
# PARAMETROS INCIALES
ruedas = 14
par = "EUR/USD"
fast, slow = 20, 60
sigma = 40

fw = 10, 20

features = [f"Cruce_{fast}_{slow}", "rsi", f"sigma_{sigma}"] # atributo dinámico
forwards = [f"fw_{i}" for i in fw] # atributo preseteado, dinámico

In [4]:
# AJUSTE DE DATA
new_labels = ['Open', "High", "Low", "Close"]

data.rename(dict(zip(data.columns, new_labels)), axis=1, inplace=True)

data = data.iloc[-1::-1]


### Indicadores tardíos

-58-

In [8]:
from apps.medias_moviles import moving_averages

moving_averages(data, "Close", window=50)

,SMA_50,EMA_50,WMA_50,DEMA_50,TRIMA_L_50,TEMA_50
2003-08-29,NaN,1.097900,NaN,1.097900,NaN,1.097900
2003-09-01,NaN,1.097084,NaN,1.096684,NaN,1.096488
2003-09-02,NaN,1.091437,NaN,1.087488,NaN,1.085041
2003-09-03,NaN,1.088776,NaN,1.083917,NaN,1.081452
2003-09-04,NaN,1.089689,NaN,1.086598,NaN,1.086051
...,...,...,...,...,...,...
2022-10-24,0.989811,0.991197,0.984235,0.972776,1.009285,0.974561
2022-10-25,0.989409,0.991409,0.984502,0.973914,1.008461,0.976519
2022-10-26,0.989216,0.992058,0.985229,0.975873,1.007654,0.979635
2022-10-27,0.988972,0.992230,0.985512,0.976844,1.006864,0.981226


-61-

LISTA DE INDICADORES POR CATEGORIA:

- Momentum (trend-following):
    - MACD
    - ADX
    - DX
    - ADXR
    - MINUS_DI, PLUS_DI, MINUS_DM, PLUS_DM (componentes del ADX)
    - SAR (parabolic SAR- Trailing Stop)
    - MOM 
    - MIDPOINT
    - MIDPRICE

<br />    

- Contrarian:
    - CCI (Commodity channel index, indica inicio y fin de tendencia)
    - AROON (Anticipador de inicio de tendencia)
    - AROONOSC (diferencia entre las curvas de AROON)
    - BBAND (bollinger band)
    - TRANGE: Proxy de "volatilidad" True Range, es un estimador del rango normal de trading o movimiento
    - ATR: Average True Range, es la media móvil del True Range, la media movil del rango de movimiento, en términos absolutos

<br />

- Osciladores trend-following:
    - WILLR (similar estocástico)
    - APO (Absolute Price oscilator)
    - PPO (Percantage Price oscilator)
    - STOCH, STOCHF: El conocido oscilador estocástico sobre el OHLC
    - RSI
    - BOP (balance of power)
    - CMO (Chande momentum oscilator) - variante rsi
    - ROC (mide variacion % respecto a n ruedas atras)
    - MFI (Money flow index) - se interpreta como el RSI pero tiene en cuenta el volumen
    - TRIX - oscilador de TEMA para n ruedas atras
    - ULTOSC (Ultimate Oscilator) - oscilador de 3 períodos, es como promediar 3 RSIs con 3 n diferentes